# Order Book Floor Price Analysis

This notebook demonstrates the order book floor price estimation algorithm.

**Production Service:** `app/services/order_book.py`

## Algorithm Overview
1. Fetch active listings for a card/variant (excluding bulk lots)
2. Filter outliers (>2σ gap from neighbors)
3. Create adaptive price buckets (width = range/√n)
4. Find deepest bucket (most liquidity)
5. Return midpoint as floor estimate with confidence score

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from sqlalchemy import text

from app.db import engine
from app.services.order_book import (
    OrderBookAnalyzer,
    OrderBookResult,
    BucketInfo,
    OrderBookConfig,
    get_order_book_analyzer,
)

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
%matplotlib inline

# Initialize analyzer
analyzer = get_order_book_analyzer()
print(f"OrderBookAnalyzer ready")
print(f"  Min bucket width: ${OrderBookConfig.MIN_BUCKET_WIDTH}")
print(f"  Max bucket width: ${OrderBookConfig.MAX_BUCKET_WIDTH}")
print(f"  Outlier threshold: {OrderBookConfig.OUTLIER_SIGMA_THRESHOLD}σ")
print(f"  Stale cutoff: {OrderBookConfig.STALE_DAYS} days")

## 1. Basic Usage

The production service handles all data loading, outlier filtering, bucketing, and confidence calculation.

In [ ]:
# Basic usage: Estimate floor for a single card
result = analyzer.estimate_floor(card_id=1)

if result:
    print(f"Floor Estimate: ${result.floor_estimate:.2f}")
    print(f"Confidence: {result.confidence:.2f}")
    print(f"Total Listings: {result.total_listings}")
    print(f"Outliers Removed: {result.outliers_removed}")
    print(f"Stale Listings: {result.stale_count}")
    print(f"\nDeepest Bucket: ${result.deepest_bucket.min_price:.2f} - ${result.deepest_bucket.max_price:.2f}")
    print(f"  Count: {result.deepest_bucket.count}")
else:
    print("Insufficient data for floor estimation")

## 2. Filter by Treatment

You can filter by treatment (foil type) to get variant-specific floor estimates.

In [ ]:
# Compare floor estimates across treatments for a card
card_id = 1  # Adjust as needed
treatments = ['Classic Paper', 'Classic Foil', 'Stonefoil']

for treatment in treatments:
    result = analyzer.estimate_floor(card_id=card_id, treatment=treatment)
    if result:
        print(f"{treatment:20} ${result.floor_estimate:>7.2f}  conf={result.confidence:.2f}  n={result.total_listings}")
    else:
        print(f"{treatment:20} Insufficient data")

## 3. Inspect Buckets

The result includes all price buckets with their counts for visualization.

In [ ]:
# Inspect buckets for a result
result = analyzer.estimate_floor(card_id=1)

if result:
    print(f"Card 1 Order Book - {len(result.buckets)} buckets")
    print(f"Floor estimate: ${result.floor_estimate:.2f} (confidence: {result.confidence:.2f})")
    print()
    
    for bucket in result.buckets:
        marker = " ← FLOOR" if bucket == result.deepest_bucket else ""
        bar = '█' * min(bucket.count, 40)
        print(f"${bucket.min_price:>7.2f} - ${bucket.max_price:<7.2f}: {bucket.count:>3} {bar}{marker}")

## 4. Bulk Analysis

Analyze multiple cards at once to compare floor estimates.

In [ ]:
# Get cards with most active listings
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT m.card_id, c.name, COUNT(*) as listing_count
        FROM marketprice m
        JOIN card c ON m.card_id = c.id
        WHERE m.listing_type = 'active'
          AND m.is_bulk_lot = FALSE
        GROUP BY m.card_id, c.name
        ORDER BY listing_count DESC
        LIMIT 10
    """))
    top_cards = result.fetchall()

print("Top 10 cards by active listings (excluding bulk lots):")
for card_id, name, count in top_cards:
    print(f"  {card_id:>3}: {name:<40} ({count} listings)")

In [ ]:
## 5. Floor Estimates for Top Cards

In [ ]:
# Analyze top cards
results = []

for card_id, name, listing_count in top_cards:
    result = analyzer.estimate_floor(card_id)
    if result:
        results.append({
            'card_id': card_id,
            'name': name[:30],
            'listings': result.total_listings,
            'floor': result.floor_estimate,
            'confidence': result.confidence,
            'bucket_depth': result.deepest_bucket.count,
            'outliers': result.outliers_removed,
            'stale': result.stale_count
        })

# Display as DataFrame
df_results = pd.DataFrame(results)
df_results.style.format({
    'floor': '${:.2f}',
    'confidence': '{:.2f}'
})

## 6. Compare to Sales Floor

Validate ask floor against actual sales data (avg of lowest 4 sales).

In [ ]:
def get_sales_floor(card_id: int, days: int = 30) -> float | None:
    """Get sales-based floor (avg of 4 lowest sales). Excludes bulk lots."""
    cutoff = datetime.now(timezone.utc) - timedelta(days=days)
    
    with engine.connect() as conn:
        result = conn.execute(text("""
            SELECT AVG(price) as floor
            FROM (
                SELECT price
                FROM marketprice
                WHERE card_id = :card_id
                  AND listing_type = 'sold'
                  AND is_bulk_lot = FALSE
                  AND COALESCE(sold_date, scraped_at) >= :cutoff
                ORDER BY price ASC
                LIMIT 4
            ) lowest
        """), {'card_id': card_id, 'cutoff': cutoff})
        row = result.fetchone()
        return float(row[0]) if row and row[0] else None

# Compare ask floor vs sales floor
comparison = []
for card_id, name, _ in top_cards:
    ask_result = analyzer.estimate_floor(card_id)
    sales_floor = get_sales_floor(card_id)
    
    if ask_result and sales_floor:
        diff_pct = ((ask_result.floor_estimate - sales_floor) / sales_floor) * 100
        comparison.append({
            'card': name[:30],
            'ask_floor': ask_result.floor_estimate,
            'sales_floor': sales_floor,
            'diff_pct': diff_pct,
            'confidence': ask_result.confidence
        })

df_comp = pd.DataFrame(comparison)
df_comp.style.format({
    'ask_floor': '${:.2f}',
    'sales_floor': '${:.2f}',
    'diff_pct': '{:+.1f}%',
    'confidence': '{:.2f}'
})

## 7. Result Serialization

The result has a `to_dict()` method for API responses.

In [ ]:
# Serialize result for API response
result = analyzer.estimate_floor(card_id=1)
if result:
    import json
    print(json.dumps(result.to_dict(), indent=2))

In [ ]:
<cell_type>markdown</cell_type>## 8. Visualization

Visualize order book depth as a horizontal bar chart.

In [ ]:
def plot_order_book(card_id: int, card_name: str = None, treatment: str = None):
    """Visualize order book depth for a card."""
    result = analyzer.estimate_floor(card_id, treatment)
    
    if not result:
        print("Insufficient data")
        return
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Create bar chart
    labels = [f"${b.min_price:.0f}-{b.max_price:.0f}" for b in result.buckets]
    counts = [b.count for b in result.buckets]
    colors = ['#7dd3a8' if b == result.deepest_bucket else '#4a4a4a' for b in result.buckets]
    
    bars = ax.barh(labels, counts, color=colors, edgecolor='white')
    
    # Add count labels
    for bar, count in zip(bars, counts):
        ax.text(bar.get_width() + 0.3, bar.get_y() + bar.get_height()/2,
                str(count), va='center', fontsize=10)
    
    ax.set_xlabel('Number of Listings')
    ax.set_ylabel('Price Range')
    
    title = card_name or f"Card {card_id}"
    treatment_str = f" ({treatment})" if treatment else ""
    ax.set_title(f'Order Book Depth - {title}{treatment_str}\n'
                 f'Floor: ${result.floor_estimate:.2f} (Confidence: {result.confidence:.2f})')
    
    plt.tight_layout()
    plt.show()

# Plot for top card
if top_cards:
    card_id, name, _ = top_cards[0]
    plot_order_book(card_id, name)

In [ ]:
<cell_type>markdown</cell_type>## 9. Edge Cases

In [ ]:
# Test edge cases
print("Edge Case Tests:")
print("=" * 50)

# Card with known outlier issues (e.g., sealed product)
result = analyzer.estimate_floor(411)
if result:
    print(f"\nCard 411 (Sealed product):")
    print(f"  Floor: ${result.floor_estimate:.2f}")
    print(f"  Confidence: {result.confidence:.2f}")
    print(f"  Outliers removed: {result.outliers_removed}")
else:
    print("\nCard 411: Insufficient data")

# Very low volume card
with engine.connect() as conn:
    result_row = conn.execute(text("""
        SELECT card_id, COUNT(*) as cnt
        FROM marketprice
        WHERE listing_type = 'active'
          AND is_bulk_lot = FALSE
        GROUP BY card_id
        HAVING COUNT(*) BETWEEN 3 AND 5
        LIMIT 1
    """))
    low_vol = result_row.fetchone()

if low_vol:
    result = analyzer.estimate_floor(low_vol[0])
    if result:
        print(f"\nLow volume card (card_id={low_vol[0]}, {low_vol[1]} listings):")
        print(f"  Floor: ${result.floor_estimate:.2f}")
        print(f"  Confidence: {result.confidence:.2f}")

# Test insufficient data
result = analyzer.estimate_floor(99999)  # Non-existent card
print(f"\nNon-existent card: {result}")

In [ ]:
<cell_type>markdown</cell_type>## 10. Multiple Card Comparison

In [ ]:
# Plot top 3 cards side by side
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, (card_id, name, _) in zip(axes, top_cards[:3]):
    result = analyzer.estimate_floor(card_id)
    
    if result and result.buckets:
        labels = [f"${b.min_price:.0f}" for b in result.buckets]
        counts = [b.count for b in result.buckets]
        colors = ['#7dd3a8' if b == result.deepest_bucket else '#4a4a4a' for b in result.buckets]
        
        ax.barh(labels, counts, color=colors, edgecolor='white')
        ax.set_title(f'{name[:20]}\nFloor: ${result.floor_estimate:.2f}')
        ax.set_xlabel('Listings')

plt.tight_layout()
plt.show()

In [ ]:
<cell_type>markdown</cell_type>## Configuration

The service uses these configurable parameters:

| Parameter | Value | Description |
|-----------|-------|-------------|
| MIN_BUCKET_WIDTH | $5.00 | Minimum bucket size |
| MAX_BUCKET_WIDTH | $50.00 | Maximum bucket size |
| OUTLIER_SIGMA_THRESHOLD | 2.0σ | Gap threshold for outlier detection |
| STALE_DAYS | 14 | Listings older than this reduce confidence |
| MIN_LISTINGS | 3 | Minimum listings required for analysis |
| DEFAULT_LOOKBACK_DAYS | 30 | Default window for active listings |

## Summary

### Production Service
The algorithm is now implemented in `app/services/order_book.py` with:

- **OrderBookAnalyzer**: Main service class
- **OrderBookResult**: Dataclass for results
- **BucketInfo**: Dataclass for price buckets
- **OrderBookConfig**: Configuration constants

### Key Features
- Automatic outlier filtering (>2σ gap detection)
- Adaptive bucket sizing (range/√n)
- Bulk lot exclusion (`is_bulk_lot = FALSE`)
- Staleness-adjusted confidence scores
- Treatment-specific floor estimates
- Sales fallback when active listings insufficient

### API Endpoint
```
GET /api/v1/cards/{card_id}/order-book
GET /api/v1/cards/{card_id}/order-book?treatment=Classic%20Paper
```

### Usage
```python
from app.services.order_book import get_order_book_analyzer

analyzer = get_order_book_analyzer()
result = analyzer.estimate_floor(card_id=123, treatment="Classic Foil")

if result:
    print(f"Floor: ${result.floor_estimate:.2f}")
    print(f"Confidence: {result.confidence:.2f}")
    print(f"Source: {result.source}")  # 'order_book' or 'sales_fallback'
```

### Completed
- [x] Port algorithm to production service
- [x] Add unit tests (17 tests passing)
- [x] Add sales fallback for sparse data (99.5% coverage)
- [x] Add API endpoint (`/cards/{id}/order-book`)

### Next Steps
- [ ] Integrate with FloorPriceService for hybrid floor logic
- [ ] Build frontend depth chart component
- [ ] Add caching for performance